In [1]:
import json
import os

In [2]:
dataset_path = 'hp/hp1.txt'

In [5]:
os.path.basename(dataset_path).split('.')[0]

'hp1'

In [4]:
def read_list_from_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    if isinstance(data, list):
        return data
    else:
        raise ValueError("The JSON file does not contain a list.")

In [7]:
data = read_list_from_json('hp/hp1_questions.json')

In [ ]:
n_list, k_list, q_list = [], [], []
for d in data:
    if d['output'] is None:
        print(f"n = {d['history_upto']}, k = {d['context']}")
        continue
    n_list.append(d['history_upto'])
    k_list.append(d['context'])
    q_list.append(d['output'])
    print(f"number of questions: {len(d['output'])}")

In [21]:
keys = ['question', 'A', 'B', 'C', 'D', 'answer', 'explanation']

In [22]:
for d in data:
    a = [list(q.keys()) == keys for q in d['output']]
    if not all(a):
        print("Fail!")

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model

/cis/home/adesilva/ashwin/research/RAG-exps/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = "Qwen/Qwen2.5-1.5B-Instruct"

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_name)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


In [4]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

In [5]:
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 1,089,536 || all params: 1,544,803,840 || trainable%: 0.0705


In [6]:
lora_config

LoraConfig(task_type='CAUSAL_LM', peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='Qwen/Qwen2.5-1.5B-Instruct', revision=None, inference_mode=False, r=8, target_modules={'v_proj', 'q_proj'}, exclude_modules=None, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', trainable_token_indices=None, loftq_config={}, eva_config=None, corda_config=None, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False)